软件开发领域中最经典的口头禅就是：“don't repeat yourself"。也就是说，任何时候当你的程序中存在高度重复（或者是通过剪辑复制）的代码时，都应该想想是否是存在更高的解决办法。简而言之，元编程就是关于创建操作源代码（比如修改、生成、包装等）的函数和类。主要技术就是使用装饰器、类装饰器和元类。不过还有一些其它技术，包括签名对象，使用exec（）执行代码以及对内部函数和类的反射技术等。

## 在函数上添加包装器

In [1]:
# 在函数上添加一个包装器，增加额外的操作处理。
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time
    """
    @wraps(func)
    def wrapper(*args,**kwargs):
        start = time.time()
        result = func(*args,**kwargs)
        end = time.time()
        print(func.__name__,end-start)
        return result
    return wrapper

In [2]:
@timethis
def countdown(n):
    """
    Counts down
    """
    while n > 0:
        n -= 1

In [3]:
countdown(10000) 

countdown 0.000579833984375


In [4]:
countdown(1000000) 

countdown 0.05694103240966797


一个装饰器就是一个函数，它接受一个函数作为参数并返回一个新的函数。当你想下面这样写：

In [5]:
@timethis
def countdown():
    pass

和跟下面这样写其实效果是一样的：

In [6]:
def countdown():
    pass
countdown = timethis(countdown)

顺便说一下，内置的装饰器比如 @staticmethod, @classmethod,@property 原理也是一样的。 例如，下面这两个代码片段是等价的：

In [7]:
class A:
    @classmethod
    def method(cls):
        pass
    
class B:
    # Equivalent definition of a class method
    def method(cls):
        pass
    method = classmethod(method)

在上面的wrapper()函数中，装饰器内部定义了一个使用`*args`和`**kwargs`来接受任意参数的函数。在这个函数里面调用了原始函数并将函数结果返回，不过你还可以添加其它额外的代码（比如计时）。然后这个新的函数包装器被作为结果返回来代替原始函数。

需要强调的是装饰器并不会修改原始函数的参数签名以及返回值。 使用 `*args` 和 `**kwargs` 目的就是确保任何参数都能适用。 而返回结果值基本都是调用原始函数 `func(*args, **kwargs)` 的返回结果，其中`func`就是原始函数。

刚开始学习装饰器的时候，会使用一些简单的例子来说明，比如上面演示的这个。 不过实际场景使用时，还是有一些细节问题要注意的。 比如上面使用 `@wraps(func) `注解是很重要的， 它能保留原始函数的元数据(下一小节会讲到)，新手经常会忽略这个细节。 接下来的几个小节我们会更加深入的讲解装饰器函数的细节问题，如果你想构造你自己的装饰器函数，需要认真看一下。



## 创建装饰器时保留函数元信息

当你写了一个装饰器作用在某个函数时，但是这个函数的重要的元信息比如名字、文档字符串、注解和参数签名都丢失了。任何时候你定义装饰器时，都应该使用`functools`库中的`@wraps`装饰器来注解底层包装函数

In [8]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time.
    """
    @wraps(func)
    def wrapper(*args,**kwrags):
        start = time.time()
        result = func(*args,**kwrags)
        end = time.time()
        print(func.__name__,end-start)
        return result
    return wrapper


In [9]:
# 下面我们使用这个被包装的函数并检查它的元信息
@timethis
def countdown(n):
    '''
    count down
    '''
    while n > 0:
        n -= 1 
        

In [10]:
countdown(1000) 

countdown 5.888938903808594e-05


In [11]:
countdown.__name__

'countdown'

In [12]:
countdown.__doc__

'\n    count down\n    '

In [13]:
countdown.__annotations__

{}

在编写装饰器的时候复制元信息是一个非常重要的部分。如果你忘记了使用 @wraps ， 那么你会发现被装饰函数丢失了所有有用的信息。

## 解除一个装饰器

In [14]:
# 假设装饰器是通过@wraps来实现的，那么你可以通过访问__wrapped__属性来访问原始函数
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print('Decorator 1')
        return func(*args,**kwargs)
    return wrapper

def decorator2(func):
    @wraps(func) 
    def wrapper(*args,**kwargs):
        print("Decorator 2")
        return func(*args,**kwargs)
    return wrapper

@decorator1
@decorator2
def add(x,y):
    return x+y

In [15]:
add(2,3) 

Decorator 1
Decorator 2


5

In [16]:
add.__wrapped__(2,3) 

Decorator 2


5

>> 最后要说的是，并不是所有的装饰器都使用了` @wraps` ，因此这里的方案并不全部适用。 特别的，内置的装饰器` @staticmethod` 和 `@classmethod` 就没有遵循这个约定 (它们把原始函数存储在属性` __func__ `中)。



## 定义一个带参数的装饰器

我们用一个例子详细阐述下接受参数的处理过程。 假设你想写一个装饰器，给函数添加日志功能，同时允许用户指定日志的级别和其他的选项。 下面是这个装饰器的定义和使用示例：

In [17]:
from functools import wraps
import logging

def logged(level,name=None,message=None):
    """
    Add logging to a fucntion. level is the logging level,name is the 
    logger name,and message is the log message. If name and message aren't
    specified,they default to the function's module and name. 
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps
        def wrapper(*args,**kwargs):
            log.log(level,logmsg)
            return func(*args,**kwargs)
        return wrapper
    return decorate

# example use
@logged(logging.DEBUG)
def add(x,y):
    return x+y

@logged(logging.CRITICAL,'example')
def spam():
    print('Spam!') 

初看起来，这种实现看上去很复杂，但是核心思想很简单。 最外层的函数 logged() 接受参数并将它们作用在内部的装饰器函数上面。 内层的函数 decorate() 接受一个函数作为参数，然后在函数上面放置一个包装器。 这里的关键点是包装器是可以使用传递给 logged() 的参数的。

## 可自定义属性的装饰器

你想写一个装饰器来包装一个函数，并且允许用户提供参数在运行时控制装饰器行为。引入一个访问函数，使用 nonlocal 来修改内部变量。 然后这个访问函数被作为一个属性赋值给包装函数。

In [18]:
from functools import wraps,partial
import logging

# Utility decorator to attach a function as an attrbution obj
def attach_wrapper(obj,func=None):
    if func is None:
        return partial(attach_wrapper,obj)
    setattr(obj,func.__name__,func)
    return func

def logged(level,name=None,message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args,**kwargs):
            log.log(level,logmsg)
            return func(*args,**kwargs)
        
        # Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
        return wrapper
    return decorate

# Example use
@logged (logging.DEBUG)
def add(x,y):
    return x + y

@logged(logging.CRITICAL,'example')
def spam():
    print('Spam!')

In [19]:
import logging

In [20]:
logging.basicConfig(level=logging.DEBUG) 

In [21]:
add(2,3) 

DEBUG:__main__:add


5

In [22]:
# Change the log message
add.set_message('Add called') 

In [23]:
add(2,3) 

DEBUG:__main__:Add called


5

In [24]:
# change the log level
add.set_level(logging.WARNING) 

In [25]:
add(2,4) 

6

这一小节的关键点在于访问函数(如 set_message() 和 set_level() )，它们被作为属性赋给包装器。 每个访问函数允许使用 nonlocal 来修改函数内部的变量。



## 带可选参数的装饰器

In [26]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__

    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)

    return wrapper

# Example use
@logged
def add(x, y):
    return x + y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!') 

>> 可以看到，@logged 装饰器可以同时不带参数或带参数。

## 利用装饰器强制函数上的函数检查

In [27]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func

        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                            )
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [28]:
@typeassert(int,int) 
def add(x,y):
    return x+y 

In [29]:
add(2,3) 

5

In [30]:
add(2,'Hello') 

TypeError: Argument y must be <class 'int'>

## 将装饰器定义为类的一部分

在类中定义装饰器很简单，但是你首先要确认他的使用方式。比如它到底是作为一个实例方法还是类方法。下面我们用例子来阐述它们的不同。

In [31]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1 (self,func):
        @wraps(func)
        def wrapper(*args,**kwargs):
            print('Decorator 1')
            return func(*args,**kwargs)
        return wrapper
    
    # Decorator as a class method
    @classmethod
    def decorator2(cls,func):
        @wraps(func)
        def wrapper(*args,**kwargs):
            print('Decorator 2')
            return func(*args,**kwargs)
        return wrapper
    

In [32]:
# As an instance method
a = A()
@a.decorator1
def spam():
    pass

# As a class method
@A.decorator2
def grok():
    pass

>> 仔细观察可以发现一个是实例调用，一个是类调用。

在类中定义装饰器初看上去好像很奇怪，但是在标准库中有很多这样的例子。 特别的，@property 装饰器实际上是一个类，它里面定义了三个方法 getter(), setter(), deleter() , 每一个方法都是一个装饰器。例如：

In [33]:
class Person:
    # create a property instance
    first_name = property()
    
    # Apply decorator methods
    @first_name.getter
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self,value):
        if not isinstance(value,str):
            raise TypeError('Expected a string')
        self._first_name = value

它为什么要这么定义的主要原因是各种不同的装饰器方法会在关联的 property 实例上操作它的状态。 因此，任何时候只要你碰到需要在装饰器中记录或绑定信息，那么这不失为一种可行方法。

在类中定义装饰器有个难理解的地方就是对于额外参数 self 或 cls 的正确使用。 尽管最外层的装饰器函数比如 decorator1() 或 decorator2() 需要提供一个 self 或 cls 参数， 但是在两个装饰器内部被创建的 wrapper() 函数并不需要包含这个 self 参数。 你唯一需要这个参数是在你确实要访问包装器中这个实例的某些部分的时候。其他情况下都不用去管它。

## 将装饰器定义为类

为了将装饰器定义成一个实例，你需要确保它实现了` __call__()` 和` __get__()` 方法。 例如，下面的代码定义了一个类，它在其他函数上放置一个简单的记录层：

In [34]:
import types
from functools import wraps

class Profiled:
    def __init__(self,func):
        wraps(func)(self)
        self.ncalls = 0
    
    def __call__(self,*args,**kwargs):
        self.ncalls += 1 
        return self.__wrapped__(*args,**kwargs)
    
    def __get__(self,instance,cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self,instance) 

In [35]:
# 你可以将让当作一个普通的装饰器来使用，在类里面或外面都可以
@Profiled
def add(x,y):
    return x+y

class Spam:
    @Profiled
    def bar(self,x):
        print(self,x) 

In [36]:
add(2,3) 

5

In [37]:
add(2,5) 

7

In [38]:
add.ncalls

2

In [39]:
s = Spam() 

In [40]:
s.bar(1) 

<__main__.Spam object at 0x104ebc240> 1


In [41]:
s.bar(2) 

<__main__.Spam object at 0x104ebc240> 2


In [42]:
s.bar(3) 

<__main__.Spam object at 0x104ebc240> 3


In [43]:
Spam.bar.ncalls

3

将装饰器定义成类通常是很简单的。但是这里还是有一些细节需要解释下，特别是当你想将它作用在实例方法上的时候。

首先，使用 functools.wraps() 函数的作用跟之前还是一样，将被包装函数的元信息复制到可调用实例中去。

其次，通常很容易会忽视上面的` __get__()` 方法。如果你忽略它，保持其他代码不变再次运行， 你会发现当你去调用被装饰实例方法时出现很奇怪的问题。

## 为了类和静态方法提供装饰器

给类或静态方法提供装饰器是很简单的，不过要确保装饰器在 @classmethod 或 @staticmethod 之前。

In [44]:
import time
from functools import wraps

# A simple decorator 
def timethis(func):
    @wraps(func) 
    def wrapper(*args,**kwargs):
        start = time.time() 
        r = func(*args,**kwargs)
        end = time.time()
        print(end-start)
        return r
    return wrapper

# Class illustrating application of the decorator to different kinds of methods
class Spam:
    @timethis
    def instance_method(self,n):
        print(self,n)
        while n > 0:
            n -= 1
            
    @classmethod
    @timethis
    def class_method(cls,n):
        print(cls,n)
        while n > 0:
            n -= 1
            
    @staticmethod
    @timethis
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1

装饰后的类和静态方法可正常工作，只不过增加了额外的计时功能：

In [45]:
s = Spam() 

In [46]:
s.instance_method(1000000) 

<__main__.Spam object at 0x104f3e0f0> 1000000
0.05745410919189453


In [47]:
Spam.class_method(1000000) 

<class '__main__.Spam'> 1000000
0.05609917640686035


In [48]:
Spam.static_method(1000000) 

1000000
0.05829882621765137


如果你把装饰器的顺序写错了就会出错。例如，假设你像下面这样写：

In [49]:
class Spam2:
    @timethis
    @staticmethod
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1

In [50]:
Spam2.static_method(100000) 

TypeError: 'staticmethod' object is not callable

问题在于 @classmethod 和 @staticmethod 实际上并不会创建可直接调用的对象， 而是创建特殊的描述器对象(参考8.9小节)。因此当你试着在其他装饰器中将它们当做函数来使用时就会出错。 确保这种装饰器出现在装饰器链中的第一个位置可以修复这个问题。

## 装饰器为被包装的函数增加参数

In [51]:
# 可以使用关键字参数来给被包装函数增加额外参数。考虑下面的装饰器：
from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args,debug=False,**kwargs):
        if debug:
            print('Calling',func.__name__)
        return func(*args,**kwargs)
    return wrapper

In [52]:
@optional_debug
def spam(a,b,c):
    print(a,b,c) 

In [53]:
spam(1,2,3) 

1 2 3


In [54]:
spam(1,2,3,debug=True) 

Calling spam
1 2 3


通过装饰器来给被包装函数增加参数的做法并不常见。 尽管如此，有时候它可以避免一些重复代码。例如，如果你有下面这样的代码：

In [55]:
def a(x, debug=False):
    if debug:
        print('Calling a')

def b(x, y, z, debug=False):
    if debug:
        print('Calling b')

def c(x, y, debug=False):
    if debug:
        print('Calling c') 

那么你可以将其重构成这样:

In [56]:
from functools import wraps
import inspect

def optional_debug(func):
    if 'debug' in inspect.getfullargspec(func).args: 
        raise TypeError('debug argument already defined')
    
    @wraps(func)
    def wrapper(*args,debug=True,**kwargs):
        if debug:
            print('Calling',func.__name__)
        return func(*args,**kwargs)
    return wrapper

@optional_debug
def a(x):
    pass

@optional_debug
def b(x,y,z):
    pass

@optional_debug
def c(x,y):
    pass

这种实现方案之所以行得通，在于强制关键字参数很容易被添加到接受 `*args` 和 `**kwargs` 参数的函数中。 通过使用强制关键字参数，它被作为一个特殊情况被挑选出来， 并且接下来仅仅使用剩余的位置和关键字参数去调用这个函数时，这个特殊参数会被排除在外。 也就是说，它并不会被纳入到` **kwargs `中去。



## 使用装饰器扩充类的功能

这种情况可能是类装饰器最好的使用场景了。例如，下面是一个重写了特殊方法 `__getattribute__ `的类装饰器， 可以打印日志：

In [57]:
def log_getattribute(cls):
    # Get the original implemetation
    orig_getattribute = cls.__getattribute__
    
    # Make a new definition
    def new_getattribute(self,name):
        print('getting:',name)
        return orig_getattribute(self,name)
    
    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls

# Example use
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass

In [58]:
a = A(42) 

In [59]:
a.x

getting: x


42

In [60]:
a.spam

getting: spam


<bound method A.spam of <__main__.A object at 0x104ebc828>>

类装饰器通常可以作为其他高级技术比如混入或元类的一种非常简洁的替代方案。 比如，上面示例中的另外一种实现使用到继承：

In [61]:
class LoggedGetattribute:
    def __getattribute__(self, name):
        print('getting:', name)
        return super().__getattribute__(name)

# Example:
class A(LoggedGetattribute):
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass

这种方案也行得通，但是为了去理解它，你就必须知道方法调用顺序、super() 以及其它8.7小节介绍的继承知识。 某种程度上来讲，类装饰器方案就显得更加直观，并且它不会引入新的继承体系。它的运行速度也更快一些， 因为他并不依赖 super() 函数。

如果你系想在一个类上面使用多个类装饰器，那么就需要注意下顺序问题。 例如，一个装饰器A会将其装饰的方法完整替换成另一种实现， 而另一个装饰器B只是简单的在其装饰的方法中添加点额外逻辑。 那么这时候装饰器A就需要放在装饰器B的前面。



## 使用元类控制实例的创建

Python程序员都知道，如果你定义了一个类，就能像函数一样的调用它来创建实例，例如：

In [62]:
class Spam:
    def __init__(self,name):
        self.name = name
        
a = Spam('Guido')
b = Spam('Diana') 

如果你想自定义这个步骤，你可以定义一个元类并自己实现 `__call__() `方法。

为了演示，假设你不想任何人创建这个类的实例：

In [63]:
class NoInstances(type):
    def __call__(self,*args,**kwargs):
        raise TypeError("can't instantiate directly")
        
# Example 
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok') 

这样的话，用户只能调用这个类的静态方法，而不能使用通常的方法来创建它的实例。例如:

In [64]:
Spam.grok(42) 

Spam.grok


In [65]:
s = Spam() 

TypeError: can't instantiate directly

现在，假如你想实现单例模式（只能创建唯一实例的类），实现起来也很简单：

In [66]:
class Singleton(type):
    def __init__(self,*args,**kwargs):
        self._instance = None
        super().__init__(*args,**kwargs)
        
    def __call__(self,*args,**kwargs):
        if self._instance is None:
            self._instance = super().__call__(*args,**kwargs)
            return self._instance
        else:
            return self._instance

# Example
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')

那么Spam类就只能创建唯一的实例了，演示如下：

In [67]:
a = Spam() 

Creating Spam


In [68]:
b = Spam() 

In [69]:
a is b

True

In [70]:
c = Spam() 

In [71]:
a is c

True

最后，假设你想创建8.25小节中那样的缓存实例。下面我们可以通过元类来实现：

In [72]:
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()

    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj

# Example
class Spam(metaclass=Cached):
    def __init__(self, name):
        print('Creating Spam({!r})'.format(name))
        self.name = name

In [73]:
a = Spam('Guido') 

Creating Spam('Guido')


In [74]:
b = Spam('Diana')

Creating Spam('Diana')


In [75]:
c = Spam('Guido')

In [76]:
a is b

False

In [77]:
a is c

True

## 捕获类的属性定义顺序

利用元类可以很容易的捕获类的定义信息。下面使用一个OrderedDict来记录描述器的定义顺序。

In [78]:
from collections import OrderedDict

# A set of descriptors for various types
class Typed:
    _expected_type = type(None)
    def __init__(self,name=None):
        self.name = name
        
    def __set__(self,instance,value):
        if not isinstance(value,self._expected_type):
            raise TypeError('Expected '+ str(self._expected_type))
        instance.__dict__[self.name] = value
        
class Integer(Typed):
    _expected_type = int
    
class Float(Typed):
    _expected_type = float
    
class String(Typed):
    _expected_type = str
    
# Metaclass that uses an OrderedDict for class body
class OrderedMeta(type):
    def __new__(cls,clsname,bases,clsdict):
        d = dict(clsdict)
        order = []
        for name,value in clsdict.items():
            if isinstance(value,Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return type.__new__(cls,clsname,bases,d)
    
    @classmethod
    def __prepare__(cls,clsname,bases):
        return OrderedDict()

在这个元类中，执行类主体时描述器的定义顺序会被一个 `OrderedDict`捕获到， 生成的有序名称从字典中提取出来并放入类属性 `_order `中。这样的话类中的方法可以通过多种方式来使用它。 例如，下面是一个简单的类，使用这个排序字典来实现将一个类实例的数据序列化为一行CSV数据：

In [79]:
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)

# Example use
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [80]:
s = Stock('GOOG',100,490.1)

In [81]:
s.name

In [82]:
s.as_csv()

'<__main__.String object at 0x104f3e1d0>,<__main__.Integer object at 0x104f3ecc0>,<__main__.Float object at 0x104f3ee48>'

## 定义有可选参数的元类


在定义类的时候，Python允许我们使用`metaclass`关键字参数来指定特定的元类。

In [83]:
from abc import ABCMeta,abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self,maxsize=None):
        pass
    
    @abstractmethod
    def write(self,data):
        pass

为了使元类支持这些关键字参数，你必须确保在 `__prepare__() `,` __new__()` 和 ``__init__()`` 方法中 都使用强制关键字参数。就像下面这样：

In [84]:
class MyMeta(type):
    # optional
    @classmethod
    def __prepare__(cls,name,bases,*,debug=False,synchronize=False):
        pass
        return super().__prepare__(name,bases)
    
    def __new__(cls,name,bases,ns,*,debug=False,synchronize=False):
        pass
        return super().__new__(cls,name,bases,ns)
    
    def __init__(self,name,bases,ns,*,debug=False,synchronize=False):
        pass
        super().__init__(name,bases,ns) 

给一个元类添加可选关键字参数需要你完全弄懂类创建的所有步骤， 因为这些参数会被传递给每一个相关的方法。` __prepare__() `方法在所有类定义开始执行前首先被调用，用来创建类命名空间。 通常来讲，这个方法只是简单的返回一个字典或其他映射对象。 `__new__() `方法被用来实例化最终的类对象。它在类的主体被执行完后开始执行。 `__init__()` 方法最后被调用，用来执行其他的一些初始化工作。

当我们构造元类的时候，通常只需要定义一个` __new__() 或 __init__() `方法，但不是两个都定义。 但是，如果需要接受其他的关键字参数的话，这两个方法就要同时提供，并且都要提供对应的参数签名。 默认的` __prepare__() `方法接受任意的关键字参数，但是会忽略它们， 所以只有当这些额外的参数可能会影响到类命名空间的创建时你才需要去定义` __prepare__()` 方法。

通过使用强制关键字参数，在类的创建过程中我们必须通过关键字来指定这些参数。

## args和kwargs的强制参数签名

对任何涉及到操作函数调用签名的问题，你都应该使用inspect模块中的的签名特性：Signature和Parameter

In [85]:
from inspect import Signature,Parameter

In [86]:
# make a signature for a func(x,y=42,*,z=None)
parms = [Parameter('x',Parameter.POSITIONAL_OR_KEYWORD),
        Parameter('y',Parameter.POSITIONAL_OR_KEYWORD,default=42),
        Parameter('z',Parameter.KEYWORD_ONLY,default=None)] 

In [87]:
sig = Signature(parms) 

In [88]:
print(sig) 

(x, y=42, *, z=None)


一旦你有了一个签名对象，你就可以使用它的 bind() 方法很容易的将它绑定到 *args 和 `**kwargs `上去。 下面是一个简单的演示：



In [89]:
def func(*args,**kwargs):
    bound_value = sig.bind(*args,**kwargs)
    for name,value in bound_value.arguments.items():
        print(name,value) 

In [90]:
# try various examples
func(1,2,z=3) 

x 1
y 2
z 3


In [91]:
func(1) 

x 1


In [92]:
func(y=2,x=1) 

x 1
y 2


下面是一个强制函数签名更具体的例子。在代码中，我们在基类中先定义了一个非常通用的 `__init__()` 方法， 然后我们强制所有的子类必须提供一个特定的参数签名。



In [93]:
from inspect import signature,Parameter

def make_sig(*names): 
    parms = [Parameter(name,Parameter.POSITIONAL_OR_KEYWORD) for name in names]
    return Signature(parms)

class Structure:
    __signature__ = make_sig()
    def __init__(self,*args,**kwargs):
        bound_values - self.__signature__.bind(*args,**kwargs)
        for name,value in bound_values.arguments.items():
            setattr(self,name,value)
            
# Example use
class stock(Structure):
    __signature__ = make_sig('name','share','price')
    
class Point(Structure):
    __signature__ = make_sig('x','y') 

In [94]:
import inspect

In [95]:
print(inspect.signature(Stock))

(name, shares, price)


In [96]:
s1 = Stock('ACME',100,490.1) 

In [97]:
s2 = Stock('ACME',100) 

TypeError: __init__() missing 1 required positional argument: 'price'

在我们需要构建通用函数库、编写装饰器或实现代理的时候，对于 *args 和 **kwargs 的使用是很普遍的。 但是，这样的函数有一个缺点就是当你想要实现自己的参数检验时，代码就会笨拙混乱。在8.11小节里面有这样一个例子。 这时候我们可以通过一个签名对象来简化它。



## 在类上强制使用编程规约

如果你想监控类的定义，通常可哟通过定义一个元类，.一个基本元类通常是继承自type并重新定义它的`__new__`方法或者是`_init__`方法。

In [98]:
class MyMeta(type):
    def __new__(self,clsname,bases,clsdict):
        # clsname is name of class being defined
        # bases is tuple of base classes
        # clsdict is class dictionary 
        return super().__new__(cls,clsname,bases,clsdict) 

In [99]:
# 另外一种定义__init__()方法
class MyMeta(type):
    def __init__(self,clsname,bases,clsdict):
        super().__init__(clsname,bases,clsdict) 

为了使用这个元类，你通常要将它放到到一个顶级父类定义中，然后其他的类继承这个顶级父类。例如：

In [100]:
class Root(metaclass=MyMeta): 
    pass

class A(Root):
    pass

class B(Root):
    pass

元类的一个关键特点是它允许你在定义的时候检查类的内容。在重新定义` __init__()` 方法中， 你可以很轻松的检查类字典、父类等等。并且，一旦某个元类被指定给了某个类，那么就会被继承到所有子类中去。 因此，一个框架的构建者就能在大型的继承体系中通过给一个顶级父类指定一个元类去捕获所有下面子类的定义。



>> 在大型面向对象的程序中，通常将类的定义放在元类中控制是很有用的。 元类可以监控类的定义，警告编程人员某些没有注意到的可能出现的问题。

